In [1]:
from pathlib import Path
import os

root_dir = Path(os.getcwd())
config_path = root_dir / 'agent' / 'config.yaml'

In [2]:
# load config
import mlflow
from src.config import parse_config

mlflow_config = mlflow.models.ModelConfig(development_config=config_path)
config = parse_config(mlflow_config)

In [3]:
from src.retriever import get_vector_retriever
retriever = get_vector_retriever(config)

from databricks_langchain import ChatDatabricks
model = ChatDatabricks(endpoint=config.model.endpoint_name)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


We are going to use Langchain's ReAct Agent with Tools

In [4]:
from databricks_langchain.uc_ai import (
    DatabricksFunctionClient,
    UCFunctionToolkit,
    set_uc_function_client,
)

client = DatabricksFunctionClient()
set_uc_function_client(client)

tools = UCFunctionToolkit(
    function_names=[
        "shm.noc_agent.vector_search",
        "shm.noc_agent.get_occupation_employment",
        "shm.noc_agent.get_occupation_wages"
        ]
).tools

In [5]:
from langgraph.prebuilt import create_react_agent
agent = create_react_agent(model, tools, prompt=config.agent.prompt)

In [14]:
input_example = {'messages':[
    {
        'type':'user', 
        'content':'I am haul truck driver in oil and gas, give me a report on my occupation over the past 12 months'
    }
]}

In [15]:
from src.utils import react_agent_to_chat_response
from langchain_core.runnables import RunnableLambda

chain = agent | RunnableLambda(react_agent_to_chat_response)
output = chain.invoke(input_example)

/Users/scott.mckean/Repos/noc_agent/.venv/lib/python3.12/site-packages/unitycatalog/ai/core/databricks.py:713: UserWarning: The following parameters do not have descriptions: occupation_description for the function shm.noc_agent.vector_search. Using Unity Catalog functions that do not have parameter descriptions limits the functionality for an LLM to understand how to call your function. To improve tool calling accuracy, provide verbose parameter descriptions that fully explain what the expected usage of the function arguments are.
  check_function_info(function_info)
/Users/scott.mckean/Repos/noc_agent/.venv/lib/python3.12/site-packages/unitycatalog/ai/core/databricks.py:713: UserWarning: The following parameters do not have descriptions: occupation_name, months_lookback for the function shm.noc_agent.get_occupation_employment. Using Unity Catalog functions that do not have parameter descriptions limits the functionality for an LLM to understand how to call your function. To improve t

## What our App needs to display at the end

In [17]:
from IPython.display import Markdown, display
display(Markdown(output['choices'][0]['message']['content']))

## Occupation Report: Haul Truck Driver in Oil and Gas
### Occupation Description
Based on the National Occupational Classification (NOC) system, a haul truck driver in the oil and gas industry falls under the category of "Trades, transport and equipment operators and related occupations, except management [72-75]". This category includes various occupations related to transportation, equipment operation, and maintenance.

### Employment Data
Over the past 12 months, the employment data for "Trades, transport and equipment operators and related occupations, except management [72-75]" has shown a relatively stable trend. The number of employees in this category has fluctuated between 3127.4 thousand in March 2024 and 3208.9 thousand in October 2024. The most recent data point available is for February 2025, with 3121.9 thousand employees.

### Wage Data
The average weekly wages for "Trades, transport and equipment operators and related occupations, except management [72-75]" have also shown a stable trend over the past 12 months. The average weekly wages have ranged from $1346.84 in July 2024 to $1387.24 in September 2024. The most recent data point available is for February 2025, with an average weekly wage of $1384.97.

### Conclusion
In conclusion, the occupation of haul truck driver in the oil and gas industry is classified under the NOC category of "Trades, transport and equipment operators and related occupations, except management [72-75]". The employment data for this category has shown a relatively stable trend over the past 12 months, with a range of 3127.4 thousand to 3208.9 thousand employees. The average weekly wages for this category have also been stable, ranging from $1346.84 to $1387.24. As of February 2025, the average weekly wage for this occupation is $1384.97.